In [1]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from io import StringIO
%matplotlib inline

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. Make use of the API to find all aggravated burglary incidents that were reported during the six month period from January 1, 2021 through June 30, 2021.

In [2]:
#get geojson from nashville.gov
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.geojson'
#params to select for aggravated burglary and dates
params = {
    'offense_description': 'BURGLARY- AGGRAVATED',
    '$where': 'incident_reported between \'2021-01-01\' and\'2021-06-30\'',
    '$limit': 2000
}
response = requests.get(endpoint, params = params)

#convert dictionary to text and convert to dataframe with geopandas
crime = (gpd.read_file(StringIO(response.text)))

#drop duplicates to prevent double counting incidents with multiple victims
crime = crime.drop_duplicates(subset = 'incident_number')

2. Download the 2019 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html. (The FIPS code for Tennessee is 47). Perform a spatial join to determine the census tract in which each burglary incident occurred. Which census tract had the highest number of burglaries? Warning - each incident can appear multiple times if there are multiple victims, so be sure that you aren't double-counting any incidents.

In [10]:
#download census tract file and place in data folder. Read the .shp in with Geopandas and assign to census_tract
census_tract = gpd.read_file('../data/tl_2019_47_tract/tl_2019_47_tract.shp')

#convert from geoseries to geodataframe and change crs type to match crime GeoDataFrame 'EPSG:4326'
census_tract = gpd.GeoDataFrame(census_tract, 
                           crs = crime.crs, 
                           geometry = crime['geometry'])

#confirm identical crs type
print(census_tract.crs)
print(crime.crs)
print(type(crime))
print(type(census_tract))

epsg:4326
epsg:4326
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>


In [7]:
#join crime with census_tract
crime_census_tract = gpd.sjoin(crime, census_tract, op = 'within')

In [8]:
crime_census_tract.head()

,victim_county_resident,zip_code,victim_number,offense_nibrs,rpa,latitude,victim_race,incident_number,investigation_status,offense_number,...,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
0,RESIDENT,None,1,220,8203,36.15,B,20210249540,Open,1,...,021002,47165021002,210.02,Census Tract 210.02,G5020,S,30654374,207994,+36.3521602,-086.6464309
158,RESIDENT,None,1,220,8153,36.15,W,20210223220,Open,1,...,021002,47165021002,210.02,Census Tract 210.02,G5020,S,30654374,207994,+36.3521602,-086.6464309
630,RESIDENT,None,1,220,8201,36.15,B,20210284867,Open,2,...,021002,47165021002,210.02,Census Tract 210.02,G5020,S,30654374,207994,+36.3521602,-086.6464309
0,RESIDENT,None,1,220,8203,36.15,B,20210249540,Open,1,...,080802,47155080802,808.02,Census Tract 808.02,G5020,S,22956501,0,+35.8454674,-083.5220092
158,RESIDENT,None,1,220,8153,36.15,W,20210223220,Open,1,...,080802,47155080802,808.02,Census Tract 808.02,G5020,S,22956501,0,+35.8454674,-083.5220092


In [11]:
crime_census_tract.columns

Index(['victim_county_resident', 'zip_code', 'victim_number', 'offense_nibrs',
       'rpa', 'latitude', 'victim_race', 'incident_number',
       'investigation_status', 'offense_number', 'offense_description',
       'longitude', 'domestic_related', 'victim_ethnicity',
       'incident_location', 'victim_description', 'location_code',
       'report_type', 'incident_reported', 'weapon_description',
       'location_description', 'weapon_primary', 'report_type_description',
       'victim_type', 'incident_status_code', 'incident_status_description',
       'zone', 'victim_gender', 'incident_occurred', 'primary_key', 'geometry',
       'index_right', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON'],
      dtype='object')